# Instructions

This notebook is meant to be a way for end-users to run the issuer researcher code in a low-effort self service fashion.

Note that as of Sept 2023, we are still rate limited by OpenAI, which causes this notebook to be quite slow as we run into their rate limits quickly. Until this is fixed, it should be expected that this notebook will take several hours to run. I recommend keeping it open in a tab and ignoring it until it says it's completed. It will run even if the tab is closed, but AFAIK, it is not possible to recover the session and be synced up with current progress once the session dies.

This notebook expects there to be an 'issuers.xlsx' excel file uploaded to the Jupyter instance. This file must have an 'issuer_name' column containing the names of all issuers to be researched. The notebook can contain any other columns, they will just be ignored.

To run the notebook, run all of the cells from "Imports" onwards. The dependencies section should only need to be re-run if additional dependencies need to be installed, or the version of langchain or openai needs to be updated for some reason.

# Dependencies

Ideally, these would be in a requirements.txt. However, this notebook is meant for researcher/end-user use, so I think it's easier for them to be here for transparency, and in the event they need to be changed or updated.

In [284]:
!pip install langchain
!pip install openai
!pip install google-search-results

# Imports (Run everything from here)

In [285]:
# API Keys etc
import config

import pandas as pd
import numpy as np

import os
import requests
from bs4 import BeautifulSoup
import re

from serpapi import GoogleSearch
from tqdm import tqdm

import re
from datetime import datetime, timedelta

# Set Up

In [286]:
# Setting env vars. These come from the config.py file. The config.py file should never be committed to git.
os.environ["OPENAI_API_KEY"] = config.OPENAI_API_KEY
os.environ["SERPAPI_API_KEY"] = config.SERPAPI_API_KEY
os.environ["APIFY_API_TOKEN"] = config.APIFY_API_TOKEN

# Scraping Functions

In [287]:
def get_google_news(issuer_name):
    # We append "company" to the search to term to avoid news about things which share names with issuers
    # but are not business related.
    params = {
        "q": issuer_name + " company",
        "tbm": "nws",
        "api_key": os.environ["SERPAPI_API_KEY"]
    }

    search = GoogleSearch(params)
    results = search.get_dict()
    return results.get("news_results", [])

In [288]:
# For context reasons, it's generally not practical to pass entire page HTML to an LLM for processing. This function
# extracts just the text from the web page.
def get_raw_text(url):
    # Send a GET request to the URL
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    response = requests.get(url, headers=headers, timeout=10)
    response.raise_for_status()  # Raise an exception if there was an error

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.text, "html.parser")

    # Extract all text from the parsed HTML
    text = soup.get_text(separator=" ")

    # Remove leading/trailing white spaces and extra line breaks
    text = text.strip()
    
    # Replace newlines and repeated newlines with a single space
    text = re.sub(r'\n+', ' ', text)
    
    # Replace duplicate spaces with a single space
    text = re.sub(r' +', ' ', text)

    return text

# LLM Functions

In [289]:
# Instantiating LLM once to make it easier to swap and test

from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model='gpt-4')

In [290]:
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)

In [291]:
# We summarize the news to use both as an output (we care about the summary), but also to minimize the number of
# tokens we use in later analysis. We pass the description instead of the full article text for all other
# model inputs.
def summarize_news(page_text):
    human_message_template = """
        Below is the raw text content from a news article about a company. Please provide a short ~50 word summary
        of the contents of the article in the style of a neutral financial analyst.
        
        Please do not use the phrase "As a neutral financial analyst."

        {content}
    """

    human_message_prompt = HumanMessagePromptTemplate.from_template(human_message_template)

    chat_prompt = ChatPromptTemplate.from_messages([human_message_prompt])

    chain = LLMChain(llm=llm, prompt=chat_prompt)

    return chain.run(content=page_text)

In [292]:
# We rate reliability so that we can filter out overly biased content. Google news left to its own devices will tend
# to return some amount of marketing material/press releases.
def rate_reliability(source, page_text):
    human_message_template = """
        Given the source of an article and its contents, rate the perceived "reliability" of the content on a
        scale of 1.0-5.0, where 1 is unreliable and 5 is very reliable. 
        
        Very reliable content would be unbiased from a reliable major source, whereas unreliable content would be
        clearly biased and from low quality sources such as industry press release syndications or similar publications.
        
        Do not return anything other than a number from 1-5.

        source: {source}
        {content}
    """

    human_message_prompt = HumanMessagePromptTemplate.from_template(human_message_template)

    chat_prompt = ChatPromptTemplate.from_messages([human_message_prompt])

    chain = LLMChain(llm=llm, prompt=chat_prompt)

    return chain.run(content=page_text, source=source)

In [293]:
# We rate sentiment to filter out articles which are overly negative. This isn't to say we're unwilling to host
# negative articles in general, but they require more human scrutiny.
def rate_sentiment(page_text):
    human_message_template = """
        Given the text of an article, rate the sentiment of the article from 1.0-5.0 with 1 being very negative and 5 being
        very positive. 3 should be neutral.
        
        Do not return any text other than the rating.

        {content}
    """

    human_message_prompt = HumanMessagePromptTemplate.from_template(human_message_template)

    chat_prompt = ChatPromptTemplate.from_messages([human_message_prompt])

    chain = LLMChain(llm=llm, prompt=chat_prompt)

    return chain.run(content=page_text)

In [294]:
# We rate relevance to help with "disambiguation" issues. Often we can get news back about companies that share a name
# with the target issuer, or even news about non business entities which share a name with the issuer. We score
# relevance to help filter out some of this news.
def rate_relevance(page_text):
    human_message_template = """
        Given the text of an article, rate the relevance of the article from 1.0-5.0 with 1 being irrelevant and 5 being
        very relevant. 3 should be neutral.
        
        Relevance should be judged according to the standards of business news. Business announcements such as partnerships,
        new products, leadership changes, funding, layoffs etc should be considered relevant. News like celebrity
        endorsements or similar should be considered less relevant unless very impactful.
        
        Relevance is also relative to the fact that the news is meant for a syndication about tech startups. If the news
        appears to not be about a tech startup, relevance should be low (less than 3.)
        
        Do not return any text other than the rating.

        {content}
    """

    human_message_prompt = HumanMessagePromptTemplate.from_template(human_message_template)

    chat_prompt = ChatPromptTemplate.from_messages([human_message_prompt])

    chain = LLMChain(llm=llm, prompt=chat_prompt)

    return chain.run(content=page_text)

In [295]:
# Controversy is a bit odd, as one would think it's related to sentiment, this ends up not being true often.
def rate_controversy(page_text):
    human_message_template = """
        Given the text of an article, rate how controversial the article is on a scale of 1.0-5.0 with 5 being the
        most controversial, and 1 being the least. Controversial articles (for our purposes) would be articles which 
        would be unsuitable for syndication on a business news site. Controversial articles would include:
        overly political articles, or articles implicating the company in some sort of criminal or civil proceeding.
        News related to criminal activity should be rated 5.
        
        Do not return any text other than the rating.
        
        Do not prefix ratings with any text such as "Rating:

        {content}
    """

    human_message_prompt = HumanMessagePromptTemplate.from_template(human_message_template)

    chat_prompt = ChatPromptTemplate.from_messages([human_message_prompt])

    chain = LLMChain(llm=llm, prompt=chat_prompt)

    return chain.run(content=page_text)

In [296]:
# We tag news, as presumably these are useful for other use cases such as detecting certain kinds of business actions.
# The list of tags in the prompt is far from comprehensive, and could certainly be added to.
def get_tags(page_text):
    human_message_template = """
        Given the text of an article, propose a list of 5-10 tags which might be applicable to the article.
        
        We primarily care about business related tags. Some examples might include:
        
        Valuation
        Stock
        IPO
        S-1
        M&A
        SPAC
        Funding Round
        Unicorn
        
        Feel free to generate other business related tags you deem relevant.
        
        Return a comma separated list of tags, with no other text.

        {content}
    """

    human_message_prompt = HumanMessagePromptTemplate.from_template(human_message_template)

    chat_prompt = ChatPromptTemplate.from_messages([human_message_prompt])

    chain = LLMChain(llm=llm, prompt=chat_prompt)

    return chain.run(content=page_text)

In [297]:
# Adjust dates - mainly get rid of all relative dates
def parse_date(date_str : str) -> str: #yyyy-mm-dd
    '''This parses dates in the format Mon-dd-yyyy or # period(s) ago
    and is only really meant to convert dates and times from the serpAPI'''
    
    #regex templates for possible formats
    regex_days = r'^(\d{1,2}) days?\b ago$'
    regex_weeks = r'^(\d{1,2}) weeks?\b ago$'
    regex_months = r'^(\d{1,2}) months?\b ago$'
    regex_hours = r'^(\d{1,2}) hours?\b ago$'

    # brute force approach - try straight cast and rely on exception handling
    # to try alternative formats
    
    try:
        x = datetime.strptime(date_str, '%b %d, %Y')
    
    except ValueError:
        # hours ago
        x = re.search(regex_hours, date_str) 
        if x:
            offset = timedelta(hours=int(x.group(1)))
            x = datetime.now() - offset
        # days ago
        x = re.search(regex_days, date_str) 
        if x:
            offset = timedelta(days=int(x.group(1)))
            x = datetime.now() - offset
        x = re.search(regex_weeks, date_str)
        if x:
            offset = timedelta(weeks=int(x.group(1)))
            x = datetime.now() - offset
        x = re.search(regex_months, date_str)
        if x:
            offset = timedelta(days=int(x.group(1)) * 30)
            x = datetime.now() - offset
        
    return x

# News Processing

In [298]:
# We expect an excel file which has an issuer_name column containing the issuers we want to find news for.
# I recently switched the date to use the SerpAPI date scraped from Google as I expected this to be more reliable
# than the date we extract via LLM. It may need some further processing to be useful for our purposes...
def get_news_for_issuers(file_path):
    issuers = pd.read_excel(file_path)["issuer_name"].tolist()
    processed_news = []

    for i in tqdm(issuers):
        news = get_google_news(i)
        print(f'Processing news for {i}')
        for n in news:
            print(f'Processing article: {n["title"]}')
            try:
                print('Fetching raw article text')
                raw = get_raw_text(n['link'])
                print('Summarizing...')
                summary = summarize_news(raw)
                print(f'Summary: {summary}')
                reliability = rate_reliability(n['source'], raw)
                print(f'Reliability Score: {reliability}')
                sentiment = rate_sentiment(summary)
                print(f'Sentiment Score: {sentiment}')
                relevance = rate_relevance(summary)
                print(f'Relevance Score: {relevance}')
                controversy = rate_controversy(summary)
                print(f'Controversy Score: {controversy}')
                tags = get_tags(summary)
                print(f'Tags: {tags}')

                dt = parse_date(n['date'])
                print(f'input date: {n["date"]}, output was {dt}')
                
                processed_news.append({
                    'issuer': i,
                    'title': n['title'],
                    'published_at': dt,
                    'link': n['link'],
                    'source': n['source'],
                    'summary': summary,
                    'reliability': reliability,
                    'relevance': relevance,
                    'sentiment': sentiment,
                    'controversy': controversy,
                    'tags': tags,

                })
            except:
                continue

    return processed_news

In [299]:
# Some post processing to coerce non-conforming data. Unfortunately LLMs can sometimes produce nonsense, which we
# don't want in the final output.
def clean_news(news):
    issuers = []
    titles = []
    published_at = []
    links = []
    sources = []
    summaries = []
    reliabilities = []
    relevances = []
    sentiments = []
    controversies = []
    tags = []

    for i in news:
        issuers.append(i['issuer'])
        titles.append(i['title'])
        published_at.append(i['published_at'])
        links.append(i['link'])
        sources.append(i['source'])
        summaries.append(i['summary'])
        reliabilities.append(i['reliability'])
        relevances.append(i['relevance'])
        sentiments.append(i['sentiment'])
        controversies.append(i['controversy'])
        tags.append(i['tags']) 
    
    news_cleaned = pd.DataFrame()
    news_cleaned['issuer'] = issuers
    news_cleaned['title'] = titles
        
    #cleaned_publish_dates = pd.to_datetime(published_at, errors='coerce')
    news_cleaned['published_at'] = published_at
    
    news_cleaned['links'] = links
    news_cleaned['sources'] = sources
    news_cleaned['summary'] = summaries
    
    cleaned_reliability = pd.to_numeric(reliabilities, errors='coerce')
    news_cleaned['reliability'] = cleaned_reliability
    
    cleaned_relevance = pd.to_numeric(relevances, errors='coerce')
    news_cleaned['relevance'] = cleaned_relevance
    
    cleaned_sentiment = pd.to_numeric(sentiments, errors='coerce')
    news_cleaned['sentiment'] = cleaned_sentiment
    
    cleaned_controversy = pd.to_numeric(controversies, errors='coerce')
    news_cleaned['controversy'] = cleaned_controversy
    return news_cleaned

In [300]:
# Filtering specifically for the marketing use case. This could be changed for other use cases.
def filter_marketing_news(news):
    reliable = news['reliability'] > 3.0
    relevant = news['relevance'] > 3.0
    positive = news['sentiment'] > 3.0
    uncontroversial = news['controversy'] < 3.0
    recent = (news['published_at'].dt.year == 2023) | news['published_at'].isnull()
    
    marketing_news = news[reliable & relevant & positive & uncontroversial & recent]
    
    return marketing_news

In [301]:
###### Change this path if using a differently named file.
news = get_news_for_issuers('issuers.xlsx')
cleaned_news = clean_news(news)
filtered_news = filter_marketing_news(cleaned_news)
filtered_news.to_csv('filtered_news.csv')

  0%|          | 0/100 [00:00<?, ?it/s]

Processing news for Voi
Processing article: The university incubator with Klarna, Voi and Instabee as alumni
Fetching raw article text
Summarizing...
Summary: The SSE Business Lab at the Stockholm School of Economics has become a breeding ground for successful startups, including Klarna, Voi, and Instabee. Over the past ten years, the incubator has supported and invested in more than 250 startups, offering office space, networking opportunities, and business coaching. The incubator recently started investing in its companies via its venture capital network, providing a €30,000 convertible note that converts to shares when a startup raises its first round of funding.

Reliability Score: 4.3
Sentiment Score: 4.5
Relevance Score: 5.0
Controversy Score: 1.0
Tags: Startups, Venture Capital, Business Incubator, Stockholm School of Economics, Business Coaching, Networking, Investment, Funding Round, Convertible Note
input date: Sep 3, 2023, output was 2023-09-03 00:00:00
Processing article: V

  1%|          | 1/100 [02:13<3:39:51, 133.25s/it]

Tags: E-Scooter, Voi, Complaints, City Council, Safety, Underage Use, Transportation, Rideshare, Bans
input date: Feb 28, 2023, output was 2023-02-28 00:00:00
Processing news for Weights &amp; Biases
Processing article: Weights & Biases, which counts OpenAI as a customer, lands $50M
Fetching raw article text
Summarizing...
Summary: Weights & Biases, a prominent AI and machine learning development platform, has secured $50 million in a strategic funding round, bringing its total raised to $250 million and valuing the company at $1.25 billion. The investment was led by ex-GitHub CEO Nat Friedman and former Y Combinator partner Daniel Gross, along with existing investors. The funds will be used to launch Prompts, a new product designed to monitor and evaluate the performance of large language models.

Reliability Score: 4.6
Sentiment Score: 4.5
Relevance Score: 5.0
Controversy Score: 1.0
Tags: AI, Machine Learning, Funding Round, Unicorn, Valuation, Strategic Investment, Product Launch, L

  2%|▏         | 2/100 [04:15<3:27:15, 126.90s/it]

...

Tags: Embezzlement, Facebook, Diversity and Inclusion, Fraud, Corporate Crime, Vendor Management, Expense Reports, Business Ethics
input date: 5 days ago, output was None
Processing news for ISN
Processing article: ISN® Releases Latest Serious Injuries and Fatalities Insights Report
Fetching raw article text
Processing article: Costco's hottest item isn't rotisserie chickens. It's $2,000 gold bars
Fetching raw article text
Summarizing...
Summary: Costco's latest hot-selling item is gold bars, which are retailing for $2,069.99. The product usually sells out hours after being posted on the company's website. Costco, which began selling gold online in September, sold more than $100 million worth of gold bars last quarter. The company's success with gold sales comes amid strong profits from the pandemic and an over 45% surge in its shares this year.

Reliability Score: 4.2
Sentiment Score: 4.1
Relevance Score: 1.0
Controversy Score: 1.0
Tags: Costco, Gold Bars, Retail, Online Sales, Pandem

 99%|█████████▉| 99/100 [3:42:00<02:19, 139.99s/it]

Processing news for Kalshi
Processing article: Predictions market Kalshi sues CFTC for blocking election contracts
Fetching raw article text
Processing article: Hollen, Merkley, Colleagues: No Gambling on Elections | U.S. Senator Chris Van Hollen of Maryland
Fetching raw article text
Summarizing...
Summary: US Senators, led by Chris Van Hollen and Jeff Merkley, have written to the Commodity Futures Trading Commission (CFTC), urging it to reject a proposal by private predictions company, Kalshi, to offer event contracts on US election outcomes. The senators believe such a move would lead to widespread gambling on election results, undermining the integrity of the electoral process and potentially letting the wealthy influence results. The CFTC has yet to make a decision on the proposal.
Reliability Score: 4.5
Sentiment Score: 2.5
Relevance Score: 4.7
Controversy Score: 4.5
Tags: Commodity Futures Trading Commission, Election Outcomes, US Senators, Kalshi, Gambling, Electoral Process, We

100%|██████████| 100/100 [3:44:06<00:00, 134.46s/it]

Tags: Error Message, Bloomberg, Website Navigation, User Confirmation, Not Applicable
input date: Sep 22, 2023, output was 2023-09-22 00:00:00
